### Feature Engineering
##### It's a simple step, extract all edges in the train.txt document. Then save all edges line by line in edges.txt


In [5]:
# Candidate0: 
weight_dict = {}
nodes = []
with open('data/train.txt', 'r') as f, open('data/edges.txt', 'w') as o, open('data/test-public.csv', 'r') as t:
    training_data = f.readlines()
    for line in training_data:
        l = line.strip().split()
        for i in range(len(l)):
            nodes.append(int(l[i]))
            for j in range(i+1, len(l)):
                if l[i] == l[j]:
                    print(line)
                    print(l[i], l[j])
                    continue
                if (int(l[i]), int(l[j])) not in weight_dict:
                    weight_dict[(int(l[i]), int(l[j]))] = 1
                else:
                    weight_dict[(int(l[i]), int(l[j]))] += 1
    #print(weight_dict)
    setOfNodes = set(nodes)
    print(len(setOfNodes))
    
    count = 0
    for key, value in weight_dict.items():
        if key[0]==1123 or key[0]==3269 or key[1]==1123 or key[1]==3269:
            print(key[0], key[1])
        if key[0] == key[1]:
            print(key[0], key[1], value)
        count += 1
        o.write(f'{key[0]} {key[1]} {value}\n')
    
    print(count)

    test_data = t.readlines()
    test_data = test_data[1:]

    for line in test_data:
        line = line.strip()
        #print(line)
        l = line.split(',')
        for i in range(1, len(l)):   # Skip the index number in the first column.
            if int(l[i]) not in setOfNodes:
                setOfNodes.add(int(l[i]))
                o.write(f'{int(l[i])} {int(l[i])} {0}\n')
                count+=1
    nodes = list(setOfNodes)
    #for node in nodes:
    #    o.write(f'{node} {node} {0.01}\n')
    print('Added', count, ' edges.')         

151 336 397 478 496 1123 1123 1319 1703 2328 3159 3322 3409 3441 3695

1123 1123
3269 3269

3269 3269
3767
151 1123
336 1123
397 1123
478 1123
496 1123
1123 1319
1123 1703
1123 2328
1123 3159
1123 3322
1123 3409
1123 3441
1123 3695
1259 3269
2557 3269
2607 3269
16034
Added 16083  edges.


In [78]:

def link_prediction_split(nodes, weight_dict, files, portions):
    """
    Divide a normal graph into a train split and several test splits for link prediction use.
    Each test split contains half true and half false edges.

    Parameters:
        graph_file (str): graph file
        files (list of str): file names,
            the first file is treated as train file
        portions (list of float): split portions
    """
    assert len(files) == len(portions)
    np.random.seed(1024)

    nodes = set(nodes)
    edges = []
    for key,_ in weight_dict.items():
        if key[0] == key[1]:
            print(key[0], key[1])
        edges.append(key)
    
    portions = np.cumsum(portions, dtype=np.float32) / np.sum(portions)
    files = [open(file, "w") for file in files]
    num_edges = [0] * len(files) # trian8， test2长度分别为多少


        

    for line in edges:
        u = line[0]
        v = line[1]
        i = np.searchsorted(portions, np.random.rand())
        if i == 0:
            if u==v:
                print(u,v)
            files[i].write(f'{u}\t{v}\t{weight_dict[(u,v)]}\n')
        else:
            if u==v:
                print(u,v)
            files[i].write(f"{u}\t{v}\t1\n")
        num_edges[i] += 1

    for node in nodes:
        files[0].write(f'{node}\t{node}\t{0.01}\n')
        
    nodes = list(nodes)
    for file, num_edge in zip(files[1:], num_edges[1:]):
        for _ in range(num_edge):
            valid = False
            while not valid:
                u = nodes[int(np.random.rand() * len(nodes))]
                v = nodes[int(np.random.rand() * len(nodes))]
                valid = u != v and (u, v) not in edges and (v, u) not in edges
            file.write(f"{u}\t{v}\t0\n")
    for file in files:
        file.close()

link_prediction_split(nodes, weight_dict, ['./data/train8.txt', './data/test2.txt'], [8, 2])

### NOTES On Colab
#### When you run your code on colab you need to load the goole's virtual drive. Run the following blocks to load drive and 'cd' to the current path.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/MyDrive/Graph_Link_Prediction/graphvite/

/content/drive/MyDrive/Graph_Link_Prediction/graphvite
